In [38]:
# 필요한 라이브러리 임포트
from contextualized_topic_models.models.ctm import CombinedTopicModel
from contextualized_topic_models.utils.preprocessing import WhiteSpacePreprocessing
from contextualized_topic_models.utils.data_preparation import TopicModelDataPreparation
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
from scipy.stats import entropy

# 데이터프레임(df)에 'Review_Text' 열이 존재해야 합니다.


ModuleNotFoundError: No module named 'contextualized_topic_models'

In [36]:
pip install contextualized_topic_models

  Using cached contextualized_topic_models-2.5.0-py2.py3-none-any.whl.metadata (24 kB)
  Using cached gensim-4.2.0.tar.gz (23.2 MB)
  Preparing metadata (setup.py) ... done
  Using cached sentence_transformers-3.3.1-py3-none-any.whl.metadata (10 kB)
  Using cached ipywidgets-7.5.1-py2.py3-none-any.whl.metadata (1.8 kB)
  Using cached ipython-8.10.0-py3-none-any.whl.metadata (5.7 kB)
  Using cached backcall-0.2.0-py2.py3-none-any.whl.metadata (2.0 kB)
  Using cached widgetsnbextension-3.5.2-py2.py3-none-any.whl.metadata (1.3 kB)
  Using cached transformers-4.47.1-py3-none-any.whl.metadata (44 kB)
  Using cached tokenizers-0.21.0-cp39-abi3-macosx_11_0_arm64.whl.metadata (6.7 kB)
  Using cached safetensors-0.4.5-cp312-cp312-macosx_11_0_arm64.whl.metadata (3.8 kB)
Using cached contextualized_topic_models-2.5.0-py2.py3-none-any.whl (36 kB)
Using cached ipython-8.10.0-py3-none-any.whl (784 kB)
Using cached ipywidgets-7.5.1-py2.py3-none-any.whl (121 kB)
Using cached sentence_transformers-3.3.

In [7]:
# Step 1: 리뷰 데이터를 전처리하고 CTM 모델에 맞는 데이터를 준비

# 리뷰 텍스트를 전처리하기 위해 WhiteSpacePreprocessing 사용
# 필요에 따라 stopword 목록을 추가할 수 있습니다.
preprocessor = WhiteSpacePreprocessing(
    df['Review_Text'].tolist(), 
    stopwords_list=None  # 커스텀 stopword 목록을 지정하려면 여기에 추가
)
data_cleaned = preprocessor.preprocess()

# BoW(단어 빈도 벡터) 생성
vectorizer = CountVectorizer(max_features=1000)  # 최대 1000개의 단어를 BoW로 사용
data_bow = vectorizer.fit_transform(data_cleaned)
data_vocab = vectorizer.get_feature_names_out()

# CTM 모델 데이터를 준비
data_ctm = TopicModelDataPreparation("paraphrase-MiniLM-L6-v2")
data_train = data_ctm.fit(data_cleaned, bow_matrix=data_bow)

# 준비된 데이터 확인 (필요시 출력)
print("전처리된 데이터 예시:", data_cleaned[:3])
print("BoW 단어 집합 크기:", len(data_vocab))


NameError: name 'WhiteSpacePreprocessing' is not defined

In [11]:
# Step 2: CTM 모델 학습

# CTM 모델 초기화
ctm_model = CombinedTopicModel(
    bow_size=len(data_vocab),  # BoW 크기
    contextual_size=768,  # MiniLM 임베딩 크기
    n_components=10,  # 토픽 수 (10개의 토픽)
    num_epochs=20,  # 학습 반복 횟수
)

# 모델 학습
ctm_model.fit(data_train)

# 학습 완료 메시지
print("CTM 모델 학습 완료!")


NameError: name 'CombinedTopicModel' is not defined

In [13]:
# Step 3: 문장 단위의 토픽 비율 계산 함수

def get_sentence_topic_proportions(text, model, data_ctm):
    """
    주어진 텍스트를 BoW와 임베딩으로 변환하고, 문장에 대한 토픽 비율을 계산
    """
    sentence_embeddings, sentence_bow = data_ctm.transform([text])
    topic_proportions = model.get_doc_topic_distribution(sentence_bow)
    return topic_proportions[0]  # 문장에 대한 토픽 비율 반환


In [15]:
# Step 4: 리뷰 단위의 토픽 비율 계산 함수

def aggregate_to_review_level(review_text, model, data_ctm):
    """
    리뷰를 문장 단위로 나누고, 각 문장에 대한 토픽 비율을 평균내어 리뷰 단위의 토픽 비율 계산
    """
    sentences = review_text.split('.')  # 기본적인 문장 분리
    sentence_proportions = []

    for sentence in sentences:
        if sentence.strip():  # 빈 문장은 건너뛰기
            sentence_prop = get_sentence_topic_proportions(sentence.strip(), model, data_ctm)
            sentence_proportions.append(sentence_prop)

    # 리뷰 전체에 대한 평균 토픽 비율 계산
    avg_topic_proportions = np.mean(sentence_proportions, axis=0)
    return avg_topic_proportions


In [17]:
# Step 5: Shannon 엔트로피 계산 함수

def calculate_entropy(topic_proportions):
    """
    주어진 토픽 비율에 대한 Shannon 엔트로피 계산
    """
    return entropy(topic_proportions, base=2)  # base=2는 정보 이론에서 일반적인 로그 기준


In [19]:
# Step 6: ContentDepth 계산 함수

def compute_content_depth(review_text, model, data_ctm):
    """
    리뷰에 대한 ContentDepth를 계산 (엔트로피를 문장 수로 정규화한 후 부호 반전)
    """
    # 리뷰 단위의 토픽 비율 계산
    avg_topic_proportions = aggregate_to_review_level(review_text, model, data_ctm)

    # 엔트로피 계산
    entropy_value = calculate_entropy(avg_topic_proportions)

    # 문장 수로 정규화
    num_sentences = len([s for s in review_text.split('.') if s.strip()])

    # ContentDepth 계산
    content_depth = -entropy_value / num_sentences if num_sentences > 0 else 0
    return content_depth


In [21]:
# Step 7: ContentDepth를 데이터프레임에 추가

df['ContentDepth'] = df['Review_Text'].apply(lambda x: compute_content_depth(x, ctm_model, data_ctm))

# 결과 확인
print(df[['Review_Text', 'ContentDepth']].head())


NameError: name 'df' is not defined